In [1]:
import pymysql as mariadb
import pandas as pd
import matplotlib
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import random
from sklearn.cluster import KMeans
import re
from wordcloud import WordCloud
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

/home/adriana/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


# Zeitreihe erstellen

Der nachfolgende Code erstellt basierend auf den gescrapten Artikel und Kommentare als auch den Daten vom BAG eine Zeitreihe.

In [2]:
#get all days as a list

from datetime import date, timedelta

sdate = date(2020, 1, 1)   # start date
edate = date(2020, 12, 31)   # end date

delta = edate - sdate       # as timedelta

kalender = []
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    kalender.append(day)

In [3]:
#establish dataframe from list
df = pd.DataFrame(kalender)
df = df.set_index(df[0]).astype(str)
df = df.rename({0: "tage"}, axis = "columns")
df

,tage
0,
2020-01-01,2020-01-01
2020-01-02,2020-01-02
2020-01-03,2020-01-03
2020-01-04,2020-01-04
2020-01-05,2020-01-05
...,...
2020-12-27,2020-12-27
2020-12-28,2020-12-28
2020-12-29,2020-12-29


In [4]:
#artikel
connection = mariadb.connect(host="localhost", database="masterarbeit",
                            user="USER", password="PASSWORD")

#filtering out the datasources where there are no comments or comments without date
sql = ("SELECT * "
       "FROM article "
       "WHERE newsoutlet NOT Like 'nau' "
       "AND newsoutlet NOT like 'nlz' "
       "AND newsoutlet NOT like 'woz' ")

df_artikel = pd.read_sql_query(sql, connection)

df_artikel_onlywithdate = df_artikel
df_artikel_onlywithdate["date"] = df_artikel_onlywithdate["art_datum"].astype(str)
df_artikel_onlywithdate = df_artikel_onlywithdate.sort_values(by="date")
df_artikel_onlywithdate = df_artikel_onlywithdate.mask(df_artikel_onlywithdate.eq('None')).dropna(subset=["date"])

df_artikel_onlywithdate = df_artikel_onlywithdate.set_index("date")

df_art_zf = df_artikel_onlywithdate.groupby(["date"]).size()
df_art_zf  = df_art_zf.to_frame()

In [7]:
#connecting to commentaires
sql2 = ("SELECT * "
        "FROM comment "
        "WHERE outlet NOT Like 'nau' ")

df_komm = pd.read_sql_query(sql2, connection)

df_komm_onlywithdate = df_komm
df_komm_onlywithdate["date"] = df_komm_onlywithdate["kom_datum"].astype(str)

df_komm_onlywithdate = df_komm_onlywithdate.sort_values(by="date")
df_komm_onlywithdate = df_komm_onlywithdate.set_index("date")

df_komm_zf = df_komm_onlywithdate.groupby(["date"]).size()
df_komm_zf = df_komm_zf.to_frame()

In [ ]:
#fallzahlen

sql3 = ("SELECT * "
       "FROM bag_fallzahlen "
       "WHERE geoRegion = 'CH'")

df_fallzahlen = pd.read_sql_query(sql3, connection)

df_fallzahlen = df_fallzahlen.sort_values(by = "datum", ascending = True)
df_fallzahlen["datum"] = df_fallzahlen["datum"].astype(str)
df_fallzahlen
df_fallzahlen = df_fallzahlen.rename(columns = {"entries": "entries_fall"})
df_fall2 = df_fallzahlen[["datum", "entries_fall"]].copy()

In [ ]:
#hospzahlen

sql3 = ("SELECT * "
       "FROM bag_hospzahlen "
       "WHERE geoRegion = 'CH'")

df_hospzahlen = pd.read_sql_query(sql3, connection)

df_hospzahlen["hosp_datum"] = pd.to_datetime(df_hospzahlen["hosp_datum"])
df_hospzahlen["hosp_datum"] = df_hospzahlen["hosp_datum"].astype(str)
df_hospzahlen = df_hospzahlen.rename(columns = {"entries": "entries_hosp"})
df_hosp2 = df_hospzahlen[["hosp_datum", "entries_hosp"]].copy()

In [ ]:
#todeszahlen

sql4 = ("SELECT * "
       "FROM bag_todzahlen "
       "WHERE geoRegion = 'CH'")

df_todzahlen = pd.read_sql_query(sql4, connection)
df_todzahlen["datum"] = pd.to_datetime(df_todzahlen["datum"], format="%d.%m.%Y")
df_todzahlen["datum"] = df_todzahlen["datum"].astype(str)
df_todzahlen = df_todzahlen.rename(columns = {"entries": "entries_tod"})
df_tod = df_todzahlen[["datum", "entries_tod"]]

In [ ]:
df_tod

In [ ]:
final = df.merge(df_art_zf, right_on = "date", left_on = "tage", how ="left")
#final = df_komm_zf

In [ ]:
final = final.merge(df_komm_zf, right_on = "date", left_on = "tage", how = "left", suffixes = (None, "_kom"))

In [ ]:
final = final.merge(df_fall2, left_on = "tage", right_on="datum", how = "left")

In [ ]:
final = final.merge(df_hosp2, left_on = "tage", right_on="hosp_datum", how="left")

In [ ]:
final = final.merge(df_tod, left_on = "tage", right_on="datum", how="left")

In [ ]:
final

In [ ]:
final = final.drop(['datum_x', 'hosp_datum', 'datum_y'], axis=1)

In [ ]:
final.to_csv("zeitreihe.csv", header = True, index = False)